In [ ]:
import pandas as pd
import geopandas as gpd

In [ ]:
valkretsar_url = pd.read_csv('req_data/valkretsar_urls.csv', header = None)
deso = gpd.read_file('output/preprocessed_data.gpkg')

In [ ]:
valresultat = pd.read_csv('req_data/rostdata.csv', delimiter = ';')
valresultat = valresultat.loc[valresultat.Distriktnamn != 'Dummy']
valresultat['Distriktkod'] = valresultat['Distriktkod'].astype(int)
for col in valresultat.loc[:, valresultat.columns.isin(['Distriktkod', 'Distriktnamn']) == False].columns:
    valresultat.loc[:, col]  = valresultat.loc[:, col].str.strip('%')
    valresultat.loc[:, col] = valresultat.loc[:, col].str.replace(',', '.')
    valresultat.loc[:, col] = valresultat.loc[:, col].astype(float)/100
    valresultat_cols.append(col.strip(''))
valresultat.columns = ['Distriktkod', 'Distriktnamn', 'S', 'SD', 'M', 'V', 'C', 'KD', 'MP', 'L']
valresultat = valresultat.loc[valresultat.Distriktnamn != 'Uppsamlingsdistrikt']

In [ ]:
gdf = gpd.GeoDataFrame()
for url in valkretsar_url.values:
    temp = gpd.read_file(url[0], crs = 'EPSG:3006')
    temp.crs = 'EPSG:3006'
    temp.set_crs(crs = 'EPSG:3006')
    gdf = pd.concat([gdf, temp])
gdf.set_crs(crs = 'EPSG:3006')
gdf.crs = 'EPSG:3006'
gdf.geometry = gdf.geometry.centroid
gdf.loc[:,'Lkfv'] = gdf['Lkfv'].astype(int)

In [ ]:
gdf = pd.merge(gdf, valresultat, left_on = 'Lkfv', right_on = 'Distriktkod')

In [ ]:
deso['poly_geom'] = deso.geometry
deso.geometry = deso.geometry.centroid
deso = deso.sjoin_nearest(gdf)
deso.geometry = deso['poly_geom']
deso.drop('poly_geom', axis = 1, inplace = True)
deso['STÖRSTA_PARTI'] = deso[['S', 'SD', 'M', 'V', 'C', 'KD', 'MP', 'L']].idxmax(axis=1)

In [ ]:
c_dict = {'S': 'r', 'SD': 'y', 'M': 'b', 'V': 'r', 'C': 'gr', 'KD': 'purple', 'MP': 

In [ ]:
deso.explore(column = 'STÖRSTA_PARTI')